In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\text_Summerizer\\text_summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [6]:
from textSummarizer.constants import *


In [7]:
class ConfriguationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestions(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config




In [8]:
import os
import urllib.request as requests
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header=requests.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )

            logger.info(f"{filename} download with the following info:\n {header}")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config=ConfriguationManager()
    data_ingestion_config=config.get_data_ingestions()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-09 18:09:41,558: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-09 18:09:41,683: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-09 18:09:41,684: INFO: common: created directory at: artifacts]
[2025-10-09 18:09:41,686: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-09 18:09:45,820: INFO: 159905081: artifacts/data_ingestion/data.zip download with the following info:
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AD87:2EF60F:135060:2BF4B6:68E7AD0F
Accept-Ranges: bytes
Date: Thu, 09 Oct 2025 12:39:44 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830026-CCU
X-Cache